In [ ]:
!pip install sentence_transformers

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive #file handling
from sentence_transformers import SentenceTransformer

In [ ]:
# check system specs

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print('Connected to a GPU')

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9

if ram_gb < 20:
  print('Not using a high-RAM runtime: {:.1f} gigabytes of available RAM'.format(ram_gb))
else:
  print('Using a high-RAM runtime: {:.1f} gigabytes of available RAM'.format(ram_gb))

Connected to a GPU
Using a high-RAM runtime: 89.6 gigabytes of available RAM


Note: shouldn't need high performance runtime

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
# note: place shortcut to shared project folder in google drive root directory
%cd /content/gdrive/MyDrive/ml2_project

/content/gdrive/.shortcut-targets-by-id/1WHLBzPq6pt_F7mh3d3goIQl4MwYlTfIh/ml2_project


In [ ]:
base_model = SentenceTransformer('./models/all-MiniLM-L12-v2')
# base_model = base_model.to(torch.device('cuda')) # use GPU

ft_final_model = SentenceTransformer('./models/finetune_mnr_final')
# ft_final_model = ft_final_model.to(torch.device('cuda')) # use GPU

In [ ]:
sds = pd.read_csv('data/small_dataset_w_embeddings.csv')
sds = sds[sds['tag'] != 'rap'].reset_index()

In [ ]:
df = pd.read_csv('data/clean_dataset_w_embeddings.csv').drop(labels = ['Unnamed: 0', 'level_0', 'Unnamed: 0.1', 'index'],
                                                             axis = 1)
df = df[df['tag'] != 'rap'].reset_index()

In [ ]:
def embedding_str_to_np(embed_as_str):
  #remove brackets and collapse whitespace
  regularized_str = embed_as_str.replace('\n ', ' ').replace("  ", " ")[1:-1]
  regularized_str = regularized_str.strip()
  np_array = np.array(regularized_str.split(" ")).astype(float)
  return(np_array)

In [ ]:
sds['embeddings_base'] = sds['embeddings_base'].map(embedding_str_to_np)
sds['embeddings_ft_final'] = sds['embeddings_ft_final'].map(embedding_str_to_np)

In [ ]:
df['embeddings_base'] = df['embeddings_base'].map(embedding_str_to_np)
df['embeddings_ft_final'] = df['embeddings_ft_final'].map(embedding_str_to_np)

Because our model is trained to match song halves, and we generally use shorter queries than half a song, our model wants to only return short songs. 

In [ ]:
def n_tokens(s):
  return len(ft_final_model.tokenizer(s)['input_ids'])
  
df["num_tokens"] = df['lyrics'].map(n_tokens)

Token indices sequence length is longer than the specified maximum sequence length for this model (1025 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
df = df[df['num_tokens'] >= 128]

In [ ]:
def cosine_similarity(v1, v2):
    d = np.dot(v1, v2)
    cos_theta = d / (np.linalg.norm(v1) * np.linalg.norm(v2))
    return(cos_theta)

In [ ]:
def relevance_scores(query_embed,df,embeddings):
    scores = [cosine_similarity(query_embed, v2) for v2 in df[embeddings]]
    scores = pd.Series(scores)
    return(scores)

In [ ]:
def semantic_search(query_sentence, df = df, model = ft_final_model, embeddings = 'embeddings_ft_final', return_top = False):
    df = df.copy()
    query_embed = model.encode(query_sentence)
    scores = relevance_scores(query_embed,df,embeddings)
    df['scores'] = scores
    sorted_df = df.sort_values(by = 'scores', ascending = False)

    #def no_newlines(s):
    #  return s.replace("\n",", ")
    #sorted_df['lyrics'] = sorted_df['lyrics'].map(no_newlines)

    if return_top == False:
        return sorted_df[['title','artist','lyrics','scores']]
    else:
        return sorted_df.iloc[0]['lyrics']

In [ ]:
pd.set_option('display.width', 100)

In [ ]:
q = "I’m happy sometimes but also sad. You wouldn’t understand"

In [ ]:
semantic_search(q, sds, base_model, 'embeddings_base')

,title,artist,lyrics,scores
417,Rewrite the Stars,Zac Efron,"\nYou know I want you\nIt's not a secret I try to hide\nI know you want me\nSo don't keep saying our hands are tied\nYou claim it's not in the cards\nAnd fate is pulling you miles away\nAnd out of reach from me\nBut you're here in my heart\nSo who can stop me if I decide\nThat you're my destiny?\n\n\nWhat if we rewrite the stars?\nSay you were made to be mine\nNothing could keep us apart\nYou'd be the one I was meant to find\nIt's up to you, and it's up to me\nNo one can say what we get to be\nSo why don't we rewrite the stars?\nMaybe the world could be ours\nTonight\n\nYou think it's easy\nYou think I don't want to run to you\nBut there are mountains\nAnd there are doors that we can't walk through\nI know you're wondering why\nBecause we're able to be\nJust you and me\nWithin these wa...",0.503007
236,Love on the Brain,Rihanna,"\nAnd you got me like, ""Oh, what you want from me?""\n(What you want from me?)\nAnd I tried to buy your pretty heart, but the price too high\nBaby, you got me like ""Oh""\nYou love when I fall apart (fall apart)\nSo you can put me together and throw me against the wall\n\n\nBaby you got me like i-i-i-i-i-i-i-i-i-hiii, woo i-i-i-i-i-i-i-i-i-hiii\nDon't you stop loving me (loving me)\nDon't quit loving me (loving me)\nJust start loving me (loving me)\nOwwwwww\n\n\nAnd babe I'm fist-fighting with fire\nJust to get close to you\nCan we burn something babe\nAnd I run for miles just to get a taste\nMust be love on the brain\nThat's got me feeling this way\nIt beats me black and blue but it fucks me so good\nAnd I can't get enough\nMust be love on the brain yeah\nAnd it keeps cursing my name\nNo...",0.447489
196,Cant Feel My Face,The Weeknd,"\n\n\nAnd I know she'll be the death of me\nAt least we'll both be numb\nAnd she'll always get the best of me\nThe worst is yet to come\nBut at least we'll both be beautiful and stay forever young\nThis I know, yeah, this I know\n\n\nShe told me, ""Don't worry about it""\nShe told me, ""Don't worry no more""\nWe both know we can't go without it\nShe told me, ""You'll never be alone"", oh, oh, woo\n\n\nI can't feel my face when I'm with you\nBut I love it, but I love it, oh\nI can't feel my face when I'm with you\nBut I love it, but I love it, oh\n\nAnd I know she'll be the death of me\nAt least we'll both be numb\nAnd she'll always get the best of me\nThe worst is yet to come\nAll the misery was necessary when we're deep in love\nThis I know (This I know), girl, I know\n\n\nShe told me, ""Don...",0.398393
375,The Weekend,SZA,"\nYou say you got a girl\nYeah, how you want me? How you want me when you got a girl?\nThe feelin' is reckless, of knowin' it's selfish\nAnd knowin' I'm desperate\nGettin' all in your love, fallin' all over love, like\nDo it 'til it hurts less\nHangin' out the back, all up in your lap\nLike is you comin' home? Is you out with her?\nI don't care, long as you're here by 10:30, no later than\nDrop them drawers, give me what I want\n\n\nMy man is my man, is your man, heard it's her man too\nMy man is my man, is your man, heard that's her man\nTuesday and Wednesday, Thursday and Friday\nI just keep him satisfied through the weekend\nYou like 9 to 5, I'm the weekend, oh-oh\nMake him lose his mind every weekend\nYou take Wednesday, Thursday, then just send him my way\nThink I got it covered f...",0.385092
33,Someone Like You,Adele,"\nI heard that you're settled down\nThat you found a girl and you're married now\nI heard that your dreams came true\nGuess she gave you things I didn't give to you\nOld friend, why are you so shy?\nAin't like you to hold back or hide from the light\n\n\nI hate to turn up out of the blue, uninvited\nBut I couldn't stay away, I couldn't fight it\nI had hoped you'd see my face\nAnd that you'd be reminded that for me, it isn't over\n\n\nNever mind, I'll find someone like you\nI wish nothing but the best for you, too\nDon't forget me, I beg\nI remember you said\nSometimes 

In [ ]:
semantic_search(q, df, ft_final_model, 'embeddings_ft_final')

,title,artist,lyrics,scores
14339,Its About Time,Young the Giant,"\nAll the kids are throwing sticks\nPolitics\nNights on the wire\nEverybody wants to get by\nIt's a test of the times\nA test of my rights\nIt's on, pretty lady\n\n\nBorn to be angry\nGrip of the vice\nClick on the trigger, girl\nSip wine on ice\nIt's about time\nBest to rewind\nYou better blow\n(Pistols, pistols, pistols)\n\n\nLights low\nGo on baby go\nIt's about time\nI'm on a tightrope\nDon't go, go\nIt's about time, it's about time\n\nEverybody come on\nLights on the beaches\nEverybody come on, delight\nClose the front door\nI don't wanna go\nLike I'm on the ""Truman Show""\nBest to let go\nYou don't wanna know\nYa better blow\n(Pistols, pistols, pistols)\n\n\nLights low\nGo on baby go\nIt's about time\nI'm on a tightrope\nDon't go, go\nIt's about time, it's about\n\n\nAll the years...",0.388912
68426,Sober,Demi Lovato,"\nI got no excuses for all of these goodbyes\nCall me when it's over 'cause I'm dying inside\nWake me when the shakes are gone\nAnd the cold sweats disappear\nCall me when it's over and myself has reappeared\n\n\nI don't know, I don't know, I don't know, I don't know why\nI do it every, every, every time\nIt's only when I'm lonely\nSometimes I just wanna cave and I don't wanna fight\nI try and I try and I try and I try and I try\nJust hold me, I'm lonely\n\n\nMomma, I'm so sorry, I'm not sober anymore\nAnd daddy, please forgive me for the drinks spilled on the floor\nTo the ones who never left me, we've been down this road before\nI'm so sorry, I'm not sober anymore\n\nI'm sorry to my future love for the man that left my bed\nFor making love the way I saved for you inside my head\nAnd ...",0.379395
81267,Under The Weather,Midnight Fusic,I know it's hard to find\nA little peace of mind\nWhen I'm stuck reminiscing again\nBut you came into my life (my life)\nAnd pushed it all aside\nYeah I thought I was going insane\n\nIt's getting hard for me to see\nA life without you in my company\nAnd now I'm trying hard to let it be\nYou make things better when im under the weather\nIt's getting hard for me to stay\nCause you don't even love me anyway\nI know I shouldn't wait another day\nOh you make things better when im under the weather\n\nCan't take it anymore\nI'm back on my bathroom floor\nThings were bright now we're fading to grey\nStayed up til half past four (past four)\nI found love in Singapore\nBut it sucks that I'm miles away\nIt's getting hard for me to see\nA life without you in my company\nAnd now I'm trying hard to...,0.374552
31707,Any Dream Will Do,Jason Donovan,"Jason Donovan\n\nI closed my eyes\nDrew back the curtain\nTo see for certain\nWhat I thought I knew\nFar, far away\nSomeone was weeping\nBut the world was sleeping\nAny dream will do\n\nI wore my coat\nWith golden lining\nBright colors shining\nWonderful and new\nAnd in the east\nThe dawn was breaking\nThe world was waking\nAny dream will do\n\nA crash of drums, a flash of light\nMy golden cloak flew out of sight\nThe colors faded into darkness\nI was left alone\nMay I return\nTo the beginning\nThe light is dimming\nAnd the dream is too\nThe world and I\nWe are still waiting\nStill hesitating\nAny dream will do\n\n(Spoken)\nAnd all of this happened\nBecause the world is waiting\nWaiting for one child\nBlack, white, yellow, no one knows\nBut a child that would grow up and turn tears to ...",0.371578
81378,Too Sad To Cry,Sasha Alex Sloan,"\nWasn't raised religious\nBut I wish that I was\nHaving nothing to believe in\nHas been killing my buzz\n\n\nYeah, I cut my hair, closed the blinds\nPlay ""Hallelujah"" like two dozen times\nAnd yesterday, I tried to pray\nBut I didn't know what to say\n\n\nI'm too sad to cry, too high to get up\nDon't even try 'cause I'm scared to fuck up\nDon't like to talk, I just lay in my bed\nDon't even try to go out with my friends\nLied to my doctor, she knew I was fakin'\nGave me some pills, but I'm too scared to take 'em\nI try and I try, but I'm too 

In [ ]:
print(semantic_search(q, df, ft_final_model, 'embeddings_ft_final', return_top = True))


Welcome to the wild, no heroes and villains
Welcome to the war we've only begun, so
Pick up your weapon and face it
There's blood on the crown; go and take it
You get one shot to make it out alive, so


Higher and higher you chase it
It's deep in your bones; go and take it
This is your moment, now is your time, so


Prove yourself and
Rise, rise
Make 'em remember you;
Rise
Push through hell and
Rise, rise
They will remember you;
Rise

Welcome to the climb up, reach for the summit
Visions pray that one false step lead the end, so


Higher and higher you chase it
It's deep in your blood; go and take it
This is your moment, take to the skies, go


Prove yourself and
Rise, rise
Make 'em remember you;
Rise
Push through hell and
Rise, rise
They will remember you;
Rise


So get along, so get along, go
Get along-long get a move on up
So get along, so get along, go
Get along-long get a move on up
And as you fight among the death beneath the dirt
(So get along, so get along, go
Get along-long g